# Language Detection on the Edge

## Imports

In [2]:
# Imports
import wave
import numpy as np
import pickle

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv1D, AvgPool1D, MaxPool1D, ZeroPadding1D, BatchNormalization, Flatten, Dense, Activation, GlobalAveragePooling1D
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

## Pre-process Data

In [9]:
def load_data_generator(dataset_dir: str, file_names: list):
    
    x_train, x_test, x_val, y_train, y_test, y_val = [], [], [], [], [], []
    
    for enum, file in enumerate(file_names):
        path = dataset_dir + file + '.pkl'
        with open(path, 'rb') as data_file:
            data = pickle.load(data_file)

            data_train = data['data_train']
            data_test = data['data_test']
            data_val = data['data_val']

        sample_rate=16000

        x_train_inter = []
        for i in data_train:
            num_sec = int(i.shape[0]/sample_rate)
            array_intermediate = i[:(num_sec*sample_rate)]
            x_train_inter.extend(np.split(array_intermediate, num_sec))
        

        x_test_inter = []
        for i in data_test:
            num_sec = int(i.shape[0]/sample_rate)
            array_intermediate = i[:(num_sec*sample_rate)]
            x_test_inter.extend(np.split(array_intermediate, num_sec))

        x_val_inter = []
        for i in data_val:
            num_sec = int(i.shape[0]/sample_rate)
            array_intermediate = i[:(num_sec*sample_rate)]
            x_val_inter.extend(np.split(array_intermediate, num_sec))

        y_train_inter = [[enum]]*len(x_train_inter)
        y_test_inter = [[enum]]*len(x_test_inter)
        y_val_inter = [[enum]]*len(x_val_inter)

        x_train.extend(x_train_inter)
        x_test.extend(x_test_inter)
        x_val.extend(x_val_inter)

        y_train.extend(y_train_inter)
        y_test.extend(y_test_inter)
        y_val.extend(y_val_inter)
        

    x_train = np.array(x_train)
    x_test = np.array(x_test)
    x_val = np.array(x_val)
    
    y_train = to_categorical(np.array(y_train))
    y_test = to_categorical(np.array(y_test))
    y_val = to_categorical(np.array(y_val))

    return x_train, x_test, x_val, y_train, y_test, y_val

In [10]:
dataset_dir = '../data/pre-train/raw/'
CLASSES = ['fleurs.de_de', 'fleurs.es_419', 'fleurs.fr_fr', 'fleurs.it_it']

x_train, x_test, x_val, y_train, y_test, y_val = load_data_generator(dataset_dir, CLASSES)

print('----------------------------------------------------------------------')
print(f'Number of samples in train: {len(x_train)}')
print(f'Number of samples in test: {len(x_test)}')
print(f'Number of samplesin validation: {len(x_val)}')
print('----------------------------------------------------------------------')
print('----------------------------------------------------------------------')
print(f'x_train shape: {x_train.shape} | y_train shape: {y_train.shape}')
print(f'x_test shape: {x_test.shape} | y_test shape: {y_test.shape}')
print(f'x_val shape: {x_val.shape} | y_val shape: {y_val.shape}')
print('----------------------------------------------------------------------')

---------------------------------------------------
Number of samples in train: 127843
Number of samples in test: 40575
Number of samplesin validation: 17137
---------------------------------------------------
---------------------------------------------------
x_train shape: (127843, 16000) | y_train shape: (127843, 4)
x_test shape: (40575, 16000) | y_test shape: (40575, 4)
x_val shape: (17137, 16000) | y_val shape: (17137, 4)
---------------------------------------------------


In [27]:
# Show distrubution of the labels!!!!!

[0. 0. 0. 1.]


In [11]:
FIXED_POINT = 9
x_train /= 2**FIXED_POINT
x_test  /= 2**FIXED_POINT

In [6]:
# perms = np.random.permutation(len(y_test))[0:250]
# x_test_250 = x_test[perms]
# y_test_250 = y_test[perms]
# np.savetxt('x_test_gsc_250.csv', x_test_250.reshape((x_test_250.shape[0], -1)), delimiter=',', fmt='%s')
# np.savetxt('y_test_gsc_250.csv', y_test_250, delimiter=',', fmt='%s')

## Build model

In [12]:
# Available devices
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print('Name of Devices: ', tf.config.list_physical_devices())

Num GPUs Available:  1
Name of Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [13]:
# Building the model

model = Sequential()
model.add(Input(shape=(16000, 1)))
model.add(Conv1D(filters = 128, kernel_size = 80, strides = 4 )) 
model.add(MaxPool1D(pool_size = 4)) 
model.add(Conv1D(filters = 128, kernel_size = 3, strides = 1 )) 
model.add(MaxPool1D(pool_size = 4)) 
model.add(Conv1D(filters = 256, kernel_size = 3, strides = 1 )) 
model.add(MaxPool1D(pool_size = 4)) 
model.add(Conv1D(filters = 512, kernel_size = 3, strides = 1 )) 
model.add(MaxPool1D(pool_size = 4)) 

model.add(Flatten())
model.add(Dense(len(CLASSES)))
model.add(Activation('softmax')) 

# EXPLORE Learning Rate
callbacks = EarlyStopping(monitor = "val_loss", patience  = 5)
opt = tf.keras.optimizers.Adam(lr=10e-4, decay = 10e-6)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

2023-04-28 18:42:14.801148: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-28 18:42:14.802668: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 3981, 128)         10368     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 995, 128)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 993, 128)          49280     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 248, 128)         0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 246, 256)          98560     
                                             

/opt/homebrew/Caskroom/miniforge/base/envs/OpenSesame/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [14]:
# Training model with GPU
with tf.device('/gpu:0'):
  model.fit(x_train, y_train, epochs=1, batch_size=64, validation_data=(x_test, y_test))

2023-04-28 18:43:10.747827: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-28 18:43:12.228659: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1998/1998 [==============================] - ETA: 0s - loss: 1.3867 - categorical_accuracy: 0.2713

2023-04-28 18:47:19.417854: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1998/1998 [==============================] - 276s 137ms/step - loss: 1.3867 - categorical_accuracy: 0.2713 - val_loss: 1.3998 - val_categorical_accuracy: 0.1646


In [15]:
with tf.device('/gpu:0'):
    model.evaluate(x_test, y_test, verbose=2)
    pred_test = model.predict(x_test)
    print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

1268/1268 - 32s - loss: 1.3997 - categorical_accuracy: 0.1646 - 32s/epoch - 25ms/step


2023-04-28 18:49:01.813486: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


tf.Tensor(
[[    0     0 10926     0]
 [    0     0 10693     0]
 [    0     0  6680     0]
 [    0     0 12276     0]], shape=(4, 4), dtype=int32)
